<a href="https://colab.research.google.com/github/SimranChhabra12/HW09/blob/main/Gesture_Drawing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install mediapipe==0.10.5 opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6
  Attempting uninstall: mediapipe
    Found existing installation: mediapipe 0.10.21
    Uninstalling mediapipe-0.10.21:
      Successfully uninstalled mediapipe-0.10.21
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.16.1 requires protobuf<6.0.0dev,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.


In [2]:
# Install current versions of necessary packages


# Import required libraries
import cv2
import mediapipe as mp
import numpy as np
from google.colab import files

# Upload a video file (e.g., a short clip of hand movements)
uploaded = files.upload()
video_filename = list(uploaded.keys())[0]

# Initialize MediaPipe Hands (using default settings)
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils

# Open the uploaded video file
cap = cv2.VideoCapture(video_filename)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Set up VideoWriter to save the output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, fps, (width, height))

# Create a blank canvas (white background) for drawing
canvas = np.ones((height, width, 3), dtype=np.uint8) * 255
last_point = None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame horizontally for a natural mirror effect
    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame to detect hands
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Get the index finger tip (landmark 8)
            h, w, _ = frame.shape
            index_finger = hand_landmarks.landmark[8]
            cx, cy = int(index_finger.x * w), int(index_finger.y * h)
            current_point = (cx, cy)

            # Draw a line on the canvas connecting previous and current points
            if last_point is not None:
                cv2.line(canvas, last_point, current_point, (255, 0, 0), thickness=4)
            last_point = current_point
    else:
        # Reset last_point if no hand is detected
        last_point = None

    # Combine the original frame and the canvas overlay
    combined = cv2.addWeighted(frame, 0.5, canvas, 0.5, 0)
    out.write(combined)

# Release resources and close windows
cap.release()
out.release()
hands.close()

print("Processing complete! Downloading the output video 'output.mp4'...")
files.download("output.mp4")


Saving IMG_4516.mp4 to IMG_4516.mp4
Processing complete! Downloading the output video 'output.mp4'...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>